# Zero-Copy Handover to PyTorch

This notebook demonstrates how to efficiently transfer data from a `synth_pdb.BatchedPeptide` object to a PyTorch `Tensor` without any memory duplication.

### Install Dependencies
If you don't have PyTorch installed, you can install it using pip:

In [ ]:
# !pip install torch

In [ ]:
import numpy as np
from synth_pdb.generator import BatchedGenerator

try:
    import torch
except ImportError:
    print("\033[91mError: PyTorch not found.\033[0m")
    print("PyTorch is required for this notebook. Please install it using: !pip install torch")

In [ ]:
# 1. Generate a batch of 10 peptides, each 5 residues long
sequence = "A" * 5
generator = BatchedGenerator(sequence_str=sequence, n_batch=10)
peptide_batch = generator.generate_batch(conformation='alpha')

# 2. Access the underlying contiguous C-ordered numpy array of coordinates
coords_np = peptide_batch.coords

In [ ]:
# 3. Create a PyTorch tensor using `torch.from_numpy()`
# This is a zero-copy operation; the tensor and numpy array share the same memory.
try:
    coords_torch = torch.from_numpy(coords_np)
    print(f"Numpy array shape: {coords_np.shape} (dtype: {coords_np.dtype})")
    print(f"PyTorch tensor shape: {coords_torch.shape} (dtype: {coords_torch.dtype})")
except NameError:
    print("Skipping PyTorch conversion due to missing dependency.")

In [ ]:
# 4. Verify that the data is shared
try:
    # Modifying the numpy array will be reflected in the torch tensor.
    coords_np[0, 0, 0] = 123.45
    # Use assert_allclose for robust shared data verification
    np.testing.assert_allclose(coords_np, coords_torch.numpy(), atol=1e-7)
    print(f"Modified value in numpy array: {coords_np[0, 0, 0]}")
    print(f"Value in PyTorch tensor: {coords_torch[0, 0, 0]}")
    print("Verification successful: Data is shared between Numpy and PyTorch.")
except NameError:
    print("Skipping verification due to missing dependency.")